In [1]:
# Package
import os
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import DatasetFolder 
from PIL import Image
from tqdm import tqdm
from models import *
from function import *
import numpy as np


!nvidia-smi
print("CUDA Available : ", torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

Wed Sep 27 15:40:46 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 537.42                 Driver Version: 537.42       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090      WDDM  | 00000000:2B:00.0  On |                  N/A |
|  0%   53C    P8              37W / 350W |    491MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
# Argument
epochs = 100
T = 1000
batch_size = 32
learning_rate = 1e-3
out_path = 'results'
reverse_path = 'caisa_xt_T1000'
if not os.path.isdir(out_path):
    os.mkdir(out_path)
    
if not os.path.isdir(reverse_path):
    os.mkdir(reverse_path)


In [3]:
# Preprocessing
test_tfm = transforms.Compose([
    transforms.Resize([160,160]),
    transforms.ToTensor(),
])
#test_tfm = transforms.Compose([
#    transforms.ToTensor(),
#])


toPIL = transforms.ToPILImage()

In [4]:
# Casia
train_set = DatasetFolder("../../dataset/casia100_dataset/casia", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)
train_set_size = int(len(train_set) * 0.8)
train_set, test_set = torch.utils.data.random_split(train_set, [train_set_size, len(train_set) - train_set_size])

#train_set = torchvision.datasets.CIFAR10(root='../../dataset/cifar10_dataset', train=True, download=True, transform=test_tfm)
#test_set = torchvision.datasets.CIFAR10(root='../../dataset/cifar10_dataset', train=False, download=True, transform=test_tfm)

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0)

Debugger :  class -> idx 
{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, '11': 11, '12': 12, '13': 13, '14': 14, '15': 15, '16': 16, '17': 17, '18': 18, '19': 19, '20': 20, '21': 21, '22': 22, '23': 23, '24': 24, '25': 25, '26': 26, '27': 27, '28': 28, '29': 29, '30': 30, '31': 31, '32': 32, '33': 33, '34': 34, '35': 35, '36': 36, '37': 37, '38': 38, '39': 39, '40': 40, '41': 41, '42': 42, '43': 43, '44': 44, '45': 45, '46': 46, '47': 47, '48': 48, '49': 49, '50': 50, '51': 51, '52': 52, '53': 53, '54': 54, '55': 55, '56': 56, '57': 57, '58': 58, '59': 59, '60': 60, '61': 61, '62': 62, '63': 63, '64': 64, '65': 65, '66': 66, '67': 67, '68': 68, '69': 69, '70': 70, '71': 71, '72': 72, '73': 73, '74': 74, '75': 75, '76': 76, '77': 77, '78': 78, '79': 79, '80': 80, '81': 81, '82': 82, '83': 83, '84': 84, '85': 85, '86': 86, '87': 87, '88': 88, '89': 89, '90': 90, '91': 91, '92': 92, '93': 93, '94': 94, '95': 95, '96': 96, '97': 97, '98': 98, '99

In [5]:
forward = ForwardProcess(T)
reverse = Unet(3, 3).to(device)
#criterion = diceLoss(3)
#criterion = nn.L1Loss()
optimizer = torch.optim.Adam(reverse.parameters(), lr=learning_rate)

for epoch in range(epochs):
    reverse.train()
    train_loss = 0
    
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        imgs, labels = batch
        imgs, labels = imgs.to(device), labels.to(device)
        timesteps = torch.randint(0, T, (imgs.size(0), ))
        #print(timesteps.shape)
        results, noise = forward.NoisePredictor(imgs, timesteps)

        output = reverse(results, timesteps.to(device))
        loss = F.l1_loss(noise, output)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        #break
    print(f'Epoch : {epoch} | Train | loss = {train_loss}')

    reverse.eval()
    
    #img = torch.randn((1, 3, imgs.size(2), imgs.size(3)), device=device)
    timesteps = torch.full((1,), T-1, device='cpu', dtype=torch.long)
    img, noise = forward.NoisePredictor(imgs[0].unsqueeze(0), timesteps)
    #print(img.shape)
    for i in range(0,T)[::-1]:
        t = torch.full((1,), i, device='cpu', dtype=torch.long)
        with torch.no_grad():
            img = forward.output_sample(reverse, img, t)
        if i % 10 == 0:
            img_tmp = img.squeeze(0)
            img_tmp = toPIL(img_tmp)
            img_tmp.save(f'{reverse_path}/img_{epoch}_{i}.png')
        
    print(f'Epoch : {epoch} | Eval | Generate successfully !')
            

100%|██████████| 399/399 [02:16<00:00,  2.93it/s]


Epoch : 0 | Train | loss = 0.004857020519700131
Epoch : 0 | Eval | Generate successfully !


100%|██████████| 399/399 [02:12<00:00,  3.01it/s]


Epoch : 1 | Train | loss = 0.002842729029210234
Epoch : 1 | Eval | Generate successfully !


100%|██████████| 399/399 [02:12<00:00,  3.02it/s]


...

Epoch : 97 | Train | loss = 0.0010515299428030761
Epoch : 97 | Eval | Generate successfully !


100%|██████████| 399/399 [02:12<00:00,  3.01it/s]


Epoch : 98 | Train | loss = 0.0010408673881198342
Epoch : 98 | Eval | Generate successfully !


100%|██████████| 399/399 [02:12<00:00,  3.01it/s]


Epoch : 99 | Train | loss = 0.0010705519268832146
Epoch : 99 | Eval | Generate successfully !


In [6]:
'''
for batch in tqdm(test_loader):
        imgs, labels = batch
        imgs, labels = imgs.to(device), labels.to(device)
        #timesteps = torch.randint(0, T, (imgs.size(0), ))
        timesteps = torch.full((imgs.size(0),), T-1, device='cpu', dtype=torch.long)
        #print(timesteps)
        results, noise = forward.NoisePredictor(imgs, timesteps)
        
        img = results[0].unsqueeze(0)
        #print(img.shape)
        for i in range(0,T)[::-1]:
            t = torch.full((1,), i, device='cpu', dtype=torch.long)
            with torch.no_grad():
                img = forward.output_sample(reverse, img, t)
            #img = torch.clamp(img, -1.0, 1.0)
            if i % 10 == 0:
                img_tmp = img.squeeze(0)
                img_tmp = toPIL(img_tmp)
                img_tmp.save(f'{reverse_path}/img_{epoch}_{i}.png')
                
'''


"\nfor batch in tqdm(test_loader):\n        imgs, labels = batch\n        imgs, labels = imgs.to(device), labels.to(device)\n        #timesteps = torch.randint(0, T, (imgs.size(0), ))\n        timesteps = torch.full((imgs.size(0),), T-1, device='cpu', dtype=torch.long)\n        #print(timesteps)\n        results, noise = forward.NoisePredictor(imgs, timesteps)\n        \n        img = results[0].unsqueeze(0)\n        #print(img.shape)\n        for i in range(0,T)[::-1]:\n            t = torch.full((1,), i, device='cpu', dtype=torch.long)\n            with torch.no_grad():\n                img = forward.output_sample(reverse, img, t)\n            #img = torch.clamp(img, -1.0, 1.0)\n            if i % 10 == 0:\n                img_tmp = img.squeeze(0)\n                img_tmp = toPIL(img_tmp)\n                img_tmp.save(f'{reverse_path}/img_{epoch}_{i}.png')\n                \n"